In [1]:
from bs4 import BeautifulSoup
import urllib2
url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=*&sort=Date&formType=Form8K&isAdv=true&stemming=true&numResults=100&fromDate=10/01/2018&toDate=12/31/2018&numResults=100"
    
content = urllib2.urlopen(url).read()
soup = BeautifulSoup(content)

D:\Softwares\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file D:\Softwares\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [2]:
def getText(url):
    content = urllib2.urlopen(url).read()
    soup = BeautifulSoup(content)

    for link in soup.find_all(id='viewFiling'):    
        if link.string[:3] == '8-K':            # Ensures that only 8-K filings, and no attachments are downloaded.
            #print link.string
            pos_arr = []                        # To find the position of the url and the company name
            a = (link.get('href'))
            for i in range(len(a)):
                if a[i] == "'":
                    pos_arr.append(i)
            url = a[pos_arr[0]+1:pos_arr[1]]
            company_name = a[pos_arr[2]+1:pos_arr[3]]
            #print company_name,url

            # Writing the text of the filing to a text file
            content2 = urllib2.urlopen(url).read()
            soup2 = BeautifulSoup(content2)
            file = open(company_name+"-8K_filing.txt","w")  
            file.write(soup2.get_text().encode('utf-8'))
            file.close()
            print 'success'
        


In [ ]:
# For all 8-K filings in Oct-Dec 18 quarter
for i in range(1,8100,100):
    if i == 1:
        url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=*&sort=Date&formType=Form8K&isAdv=true&stemming=true&numResults=100&fromDate=10/01/2018&toDate=12/31/2018&numResults=100"
    else:
        print i
        url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=*&sort=Date&startDoc="+ str(i) + "&numResults=100&isAdv=true&formType=Form8K&fromDate=10/01/2018&toDate=12/31/2018&stemming=true"
    getText(url)
    

In [ ]:
# Downloading all 8-K filings in a quarter having item 2.01. 
# Modify dates in url and decide range of i based on no of search results
for i in range(1,1400,100):
    if i == 1:
        url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=2.01&sort=Date&formType=Form8K&isAdv=true&stemming=false&numResults=100&fromDate=07/01/2018&toDate=09/30/2018&numResults=100"
    else:
        print i
        url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=2.01&sort=Date&startDoc="+str(i)+"&numResults=100&isAdv=true&formType=Form8K&fromDate=07/01/2018&toDate=09/30/2018&stemming=false"
    getText(url)

In [82]:
# Using readlines(). If we find "Completion of Acquisition or Disposition of Assets" in a line, 
# the next element in text[] will hopefully have a mention of the target company

path='Data\\Item 2.01 filings\\2018_Q4\\'
import os
for filename in os.listdir(path):
    print 'filename = ',filename,'\n'
    with codecs.open(path+filename, 'r', 'ascii','ignore') as g:
        text = g.readlines()
        for j in range(len(text)):
            if "Completion of Acquisition or Disposition of Assets" in text[j]:
                print text[j]
                for k in range(j+1,len(text)):
                    if len(text[k])>10:         # To ensure that cases where text[j] == '\n\r' and the like are eliminated
                        print text[k]
                        break
                break
    print '-------------------------------'

'''                
This fails if we have the following as in the 5th 8-K filing:
Item 2.01 Completion of Acquisition or Disposition of Assets.
On December 4, 2018, Adtalem Global Education Inc. (Adtalem) closed the sale of its ownership of all the outstanding
        equity interests in U.S. Education Holdings LLC, the holding company of its Carrington College subsidiaries, to San Joaquin Valley College, Inc.
'''
'''
Some 8-K filings are just 1 block, there are no newlines between paragraphs. Look at Adobe's 8-K below. It's the 4th one.
'''

filename =  Aclaris Therapeutics Inc-8K_filing.txt 

Item 2.01.Completion of Acquisition or Disposition of Assets.

On November 30, 2018, the Company closed its previously announced acquisition of the worldwide rights to RHOFADE (oxymetazoline hydrochloride) cream, 1%, which includes an exclusive license to the Rhofade Licensed Patents pursuant to the License Agreement, as well as additional intellectual property (the Acquisition), from Allergan Sales, LLC (Allergan), pursuant to the terms of an Asset Purchase Agreement (as amended, the APA), dated as of October 15, 2018, by and between the Company and Allergan. Reference is made to the information regarding the Acquisition and the APA contained in Item 1.01 of the Companys Current Report on Form 8-K filed with the Securities and Exchange Commission (the SEC) on October 15, 2018, which is incorporated herein by reference. 

-------------------------------
filename =  ACURA PHARMACEUTICALS INC-8K_filing.txt 

---------------------------

filename =  CorEnergy Infrastructure Trust Inc-8K_filing.txt 

UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWASHINGTON, DC 20549FORM 8-KCURRENT REPORT PURSUANT TOSECTION 13 OR 15(D) OF THESECURITIES EXCHANGE ACT OF 1934Date of report (Date of earliest event reported):December 21, 2018CorEnergy Infrastructure Trust, Inc.(Exact Name of Registrant as Specified in Its Charter)Maryland1-3329220-3431375(State or Other Jurisdiction of Incorporation)(Commission File Number)(IRS Employer Identification No.)1100 Walnut, Suite 3350, Kansas City, MO64106(Address of Principal Executive Offices)(Zip Code)(816) 875-3705(Registrants Telephone Number, Including Area Code)Not Applicable(Former Name or Former Address, if Changed Since Last Report)Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions:Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425)Soli

-------------------------------
filename =  Life On Earth Inc-8K_filing.txt 

-------------------------------
filename =  LIGAND PHARMACEUTICALS INC-8K_filing.txt 

UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWASHINGTON, D.C. 20549______________FORM 8K______________CURRENT REPORTPursuant to Section 13 or 15(d) of theSecurities Exchange Act of 1934Date of Report (Date of earliest event reported): October 10, 2018LIGAND PHARMACEUTICALS INCORPORATED (Exact Name of Registrant as Specified in Its Charter)Delaware001-3309377-0160744(State or other jurisdiction of(Commission File Number)(I.R.S. Employerincorporation or organization)Identification No.)3911 Sorrento Valley Boulevard, Suite 11092121San Diego, CA(Zip Code)(Address of principal executive offices)(858)550-7500(Registrants Telephone Number, Including Area Code)N/A(Former Name or Former Address, if Changed Since Last Report)Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obli


-------------------------------
filename =  SERVICEMASTER GLOBAL HOLDINGS INC-8K_filing.txt 

Item 2.01. Completion of Acquisition or Disposition of Assets.

On October1, 2018, the Company completed the Distribution and the Separation of Frontdoor from the Company. Frontdoor is now an independent public company trading under the symbol FTDR on the Nasdaq Global Select Market. The Distribution was made to the Companys stockholders of record as of the close of business on September14, 2018 (the Record Date), and such stockholders received one share of Frontdoor common stock for every two shares of Company common stock held as of close of business on the Record Date.

-------------------------------
filename =  SIGA TECHNOLOGIES INC-8K_filing.txt 

Completion of Acquisition or Disposition of Assets.

The disclosure regarding the Asset Purchase Agreement contained in Item 1.01 of this Current Report on Form 8-K is hereby

-------------------------------
filename =  Simlatus Corp-8K_filing

"\nSome 8-K filings are just 1 block, there are no newlines between paragraphs. Look at Adobe's 8-K below. It's the 4th one.\n"

In [74]:
# Using read()
# This assumes that the target company gets mentioned in the next 8 lines(1000 characters) after the mention of "Completition ..."

# specify path of downloaded data
path='Data\\Item 2.01 filings\\2018_Q4\\'
import os
for filename in os.listdir(path):
    print 'filename = ',filename
    with codecs.open(path+filename, 'r', 'ascii','ignore') as g:
        text = g.read()        
        start_phrase = "Completion of Acquisition or Disposition of Assets"
        if  start_phrase in text:
            start_index = text.index(start_phrase)
            #print start_index
            print text[start_index:start_index+1000]
    print '--------------------------------------------------'
'''
This has a disadvantage that multiple newlines(\n) show up and picks up extra unrequired strings, 
but atleast we increase the chance of finding target company mention
'''

filename =  Aclaris Therapeutics Inc-8K_filing.txt
Completion of Acquisition or Disposition of Assets.





On November 30, 2018, the Company closed its previously announced acquisition of the worldwide rights to RHOFADE (oxymetazoline hydrochloride) cream, 1%, which includes an exclusive license to the Rhofade Licensed Patents pursuant to the License Agreement, as well as additional intellectual property (the Acquisition), from Allergan Sales, LLC (Allergan), pursuant to the terms of an Asset Purchase Agreement (as amended, the APA), dated as of October 15, 2018, by and between the Company and Allergan. Reference is made to the information regarding the Acquisition and the APA contained in Item 1.01 of the Companys Current Report on Form 8-K filed with the Securities and Exchange Commission (the SEC) on October 15, 2018, which is incorporated herein by reference. 





At the closing of the Acquisition, the Company paid total cash consideration of approximately $66.1 million, consisti

Completion of Acquisition or Disposition of Assets.On November6, 2018, Hines Global REIT Campus at Playa Vista, a subsidiary of Hines Global REIT, Inc. (Hines Global or the Company), entered into an amendment to the Agreement of Sale and Purchase with Hart Campus Playa Vista LLC (the Purchaser), in which the purchase price was amended to be $330.1 million, exclusive of transaction costs and closing prorations, of which $10.0 million is contingent on certain leasing conditions having been met on or before November 15, 2019.  Also, on November6, 2018, the Seller sold Campus at Playa Vista to the Purchaser.  The Purchaser is not affiliated with Hines Global or its affiliates.Also, on November7, 2018, Hines Global REIT Bourke Trust, Hines Global REIT Brookes Trust and Hines Global REIT Ann Trust, all of which are subsidiaries of the Company, sold (i) 818 Bourke Street, (ii) 100 Brookes Street, and (iii) 825 Ann Street to Centuria Property Funds No. 2 Limited, a responsible entity of Centur

--------------------------------------------------
filename =  UNITED NATURAL FOODS INC-8K_filing.txt
Completion of Acquisition or Disposition of Assets.

On October22, 2018, United Natural Foods,Inc. (UNFI), completed the previously announced acquisition contemplated by that certain Agreement and Plan of Merger, as amended, (the Merger Agreement) by and among UNFI, SUPERVALU INC. (SUPERVALU), SUPERVALU Enterprises,Inc. and Jedi Merger Sub,Inc., a wholly owned subsidiary of UNFI (Merger Sub). Pursuant to the Merger Agreement, Merger Sub merged with SUPERVALU, with SUPERVALU continuing as the surviving corporation and a wholly owned subsidiary of UNFI (the Acquisition).

At the effective time of the Acquisition (the Effective Time), each share of SUPERVALU common stock, par value $0.01 per share, issued and outstanding immediately prior to the Effective Time (other than shares held by SUPERVALU as treasury stock, shares held by UNFI or Merger Sub immediately prior to the Effective Time 

'\nThis has a disadvantage that multiple newlines(\n) show up and picks up extra unrequired strings, \nbut atleast we increase the chance of finding target company mention\n'